In [1]:
import numpy as np
import pandas as pd
import calendar
import datetime
import xlsxwriter

In [2]:
data = pd.read_excel('C:\\Users\\Katalin\\Documents\\Downloads2\\MNF_UNIT_PACK 19.02.xlsx', sheet_name = 'Czifra Orsolya MNF_UNIT_PACK')

In [24]:
def molecule_table(mol):
    global pivot
    df = data[data['Molecule'] == mol]
    by_brand = df.groupby('Brand').sum()
    columns_list = by_brand.columns
    new_columns = []
    for column in columns_list:
        month = datetime.date(column.year, column.month, 1).strftime('%b')
        year = str(column.year)
        new_columns.append(month + " " + year)
    by_brand.columns = new_columns
    by_brand = by_brand.iloc[:, -13:]
    total = by_brand.sum(numeric_only=True)
    total.name = mol
    pivot = by_brand.append(total)
    last = pivot.columns[-1]
    pivot.sort_values(by = [last], axis = 0, ascending = False, inplace = True)
    pivot = pivot[pivot[last] != 0]
    return pivot

def brand_table(brand):
    br = data[data['Brand'] == brand].groupby('Brand Pack').sum()
    columns_list_br = br.columns
    new_col_br = []
    for column in columns_list_br:
        month = datetime.date(column.year, column.month, 1).strftime('%b')
        year = str(column.year)
        new_col_br.append(month + " " + year)
    br.columns = new_col_br
    br = br.iloc[:, -13:]
    total_br = br.sum(numeric_only=True)
    total_br.name = brand
    pivot_br = br.append(total_br)
    last_br = pivot_br.columns[-1]
    pivot_br.sort_values(by = [last_br], axis = 0, ascending = False, inplace = True)
    pivot_br = pivot_br[pivot_br[last_br] != 0]
    return pivot_br

def chart_addition(offset, row_plus, position):
    chart2 = workbook.add_chart({'type': 'line'})

    # Configure the series of the chart from the dataframe data.
    for i in range(len(cat_1)-offset):
        row = i + row_plus
        chart2.add_series({
            'name': [molecule, row, 0],
            'categories': [molecule, 0, 1, 0, 14],
            'values':     [molecule, row, 1, row, 14],
        })

    chart2.set_legend({'position': 'bottom'})
    chart2.set_x_axis({'num_font':  {'rotation': -90}})

    # Insert the chart into the worksheet.
    worksheet.insert_chart(position, chart2, {'x_scale': 1.2, 'y_scale': 1.1})

In [50]:
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

molecules = ['SIMVASTATIN', 'CETIRIZINE', 'PENTOXIFYLLINE', 'NEBIVOLOL', \
             'CLOPIDOGREL', 'DOMPERIDONE', 'DORZOLAMIDE', 'LATANOPROST', 'MELOXICAM', 'ZOLPIDEM', 'ACETYLSALICYLIC ACID', \
             'CYANOCOBALAMIN',  'DESLORATADINE', 'DROTAVERINE', 'AMITRIPTYLINE', 'ACECLOFENAC','DEXAMETHASONE, TOBRAMYCIN']

product_portfolio = ['SIMVEP', 'CETIRIZIN-EP', 'PENTOXYL-EP', 'PENTOXYL-EP', 'NEBIVEP', 'CLOPIDEP', 'DOMPERIDON-EP', \
                     'DORZOLEP', 'DORZOLEP KOMB', 'ERIGON', 'LAPROSEP', 'MELOXEP', 'ZOLEP', 'ACIZALEP', 'NEUROMULTIVIT-EP', \
                     'DESLOREP', 'DROTAVEP', 'TEPERIN EP', 'AKLOFEP', 'TODEP', 'OPTARIDIN', 'VENOPROTEP']

for molecule in molecules:
    molecule_table(molecule)
    cat_1 = pivot.index
    
    pivot.to_excel(writer, sheet_name=molecule)
    workbook  = writer.book
    worksheet = writer.sheets[molecule]
    
    #Row formating
    cell_format = workbook.add_format()
    cell_format.set_bg_color('#59cc51')
    bold = workbook.add_format({'bold': True})
    
    for i in pivot.index:
        if i in product_portfolio:
            worksheet.conditional_format(pivot.index.get_loc(i)+1, 0, pivot.index.get_loc(i)+1, 14, {'type': 'no_errors',
                                          'format': bold})
    
    highlight = ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2', 'J2', 'K2', 'L2', 'M2', 'N2']
    for i in highlight:
        worksheet.conditional_format(i, {'type': 'no_errors',
                                          'format': cell_format})
        

    worksheet.set_column('A:A', 30)

    # Create a chart object.
    chart = workbook.add_chart({'type': 'line'})

    # Configure the series of the chart from the dataframe data.
    chart.add_series({'name':[molecule, 1, 0], 
                      'categories': [molecule, 0, 1, 0, 14], 
                      'values': [molecule, 1, 1, 1, 14], 
                      'labels': {'series_name': True, 
                                 'position': 'center'}})
    chart.set_legend({'position': 'bottom'})
    chart.set_title ({'name': molecule})
    chart.set_x_axis({'num_font':  {'rotation': -90}})

    # Insert the chart into the worksheet.
    worksheet.insert_chart('A20', chart, {'x_scale': 1.2, 'y_scale': 1.1})

    chart_addition(1, 2, 'I20')
    
    #3rd graph
    products = ['SIMVEP', 'NEBIVEP', 'CLOPIDEP', 'ACIZALEP', 'NEUROMULTIVIT-EP', 'DESLOREP', 'ZOLEP', 'DROTAVEP']
    
    for product in pivot.index:
        if product in products:
            
            loc = pivot.index.get_loc(product)
            chart_addition(loc, loc+1, 'S20')

            
    if molecule == 'LATANOPROST':
        molecule_table('LATANOPROST, TIMOLOL')
        pivot.to_excel(writer, sheet_name=molecule, startrow=10)
        
        chart_addition(1, 12, 'S20')
        
        chart_addition(2, 13, 'A37')

     
    if molecule == 'ZOLPIDEM':
        brand_table('ZOLEP').to_excel(writer, sheet_name=molecule, startrow=10)
        chart6 = workbook.add_chart({'type': 'line'})
        for i in range(len(brand_table('ZOLEP').index)-1):
            row = i + len(cat_1) + 4
            chart6.add_series({
                'name': [molecule, row, 0],
                'categories': [molecule, 0, 1, 0, 14],
                'values':     [molecule, row, 1, row, 14]
            })

        chart6.set_legend({'position': 'bottom'})
        chart6.set_x_axis({'num_font':  {'rotation': -90}})
        worksheet.insert_chart('A37', chart6, {'x_scale': 1.2, 'y_scale': 1.1})
        
    if molecule == 'ACECLOFENAC':
        
        chart_addition(2, 3, 'S20')
        
    
    if molecule == 'DEXAMETHASONE, TOBRAMYCIN':
        
        chart_addition(2, 3, 'S20')
       

In [51]:
#Missing worksheets
molecule = 'PENTOXIFYLLINE'
molecule_table('PENTOXIFYLLINE')
cat_1 = pivot.index

pivot.to_excel(writer, sheet_name= 'PENTOXIFYLLINE - AMP')
workbook  = writer.book
worksheet = writer.sheets['PENTOXIFYLLINE - AMP']

#Row formating
cell_format = workbook.add_format()
cell_format.set_bg_color('#59cc51')
bold = workbook.add_format({'bold': True})

for i in pivot.index:
    if i in product_portfolio:
        worksheet.conditional_format(pivot.index.get_loc(i)+1, 0, pivot.index.get_loc(i)+1, 14, {'type': 'no_errors',
                                      'format': bold})

highlight = ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2', 'J2', 'K2', 'L2', 'M2', 'N2']
for i in highlight:
    worksheet.conditional_format(i, {'type': 'no_errors',
                                      'format': cell_format})


worksheet.set_column('A:A', 30)

# Create a chart object.
chart = workbook.add_chart({'type': 'line'})

# Configure the series of the chart from the dataframe data.
chart.add_series({'name':['PENTOXIFYLLINE - AMP', 1, 0], 
                  'categories': ['PENTOXIFYLLINE - AMP', 0, 1, 0, 14], 
                  'values': ['PENTOXIFYLLINE - AMP', 1, 1, 1, 14], 
                  'labels': {'series_name': True, 
                             'position': 'center'}})
chart.set_legend({'position': 'bottom'})
chart.set_title ({'name': 'PENTOXIFYLLINE'})
chart.set_x_axis({'num_font':  {'rotation': -90}})

# Insert the chart into the worksheet.
worksheet.insert_chart('A20', chart, {'x_scale': 1.2, 'y_scale': 1.1})

chart_addition(1, 2, 'I20')

In [52]:
writer.save()